In [1]:
import os
import warnings
import random
from os.path import join
import torch
import numpy as np
import pandas as pd
from scipy import sparse

In [2]:
torch.__version__

'1.9.0'

In [3]:

from utils import load_data_sequential
from utils import eval_sequential
import warnings
import random
from os.path import join
import torch
import numpy as np

warnings.filterwarnings('ignore')


def seed_everything(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

seed = 1
seed_everything(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

top_k = 20

Using backend: pytorch


In [4]:
data = '/Desktop/usc/CSCI - 566/project/data/spotify/challenge_set.json'

In [5]:
import json

with open('../challenge_set.json', 'r') as f:
  data = json.load(f)

In [6]:
f = open('../freq_song_spotify.txt', 'r')
data = f.read()
f.close()

data = data.split("\n")
data = [i.split("\t") for i in data[:-1]]

track2id = dict()
id2track = dict()

for i in data:
    track2id[i[2]] = int(i[-1])
    id2track[int(i[-1])] = i[2]

In [7]:
len(track2id)

17835

In [8]:
import json

with open('../challenge_set.json', 'r') as f:
    data = json.load(f)

In [9]:
playlists = []

for playlist in data["playlists"]:
    if len(playlist['tracks'])>2:
        pl = []
        for track in playlist['tracks']:
            if track['track_uri'] in track2id:
                pl.append(track2id[track['track_uri']])
            
        playlists.append(pl)
        
playlists = [i for i in playlists if len(i)>2]
        

num_users = len(playlists)
num_items = len(track2id)

print(f"# of users: {num_users},  # of items: {num_items}")

music_data = dict()
for idx, i in enumerate(playlists):
    music_data[idx] = i
    
    
user_train_music = dict()
user_valid_music = dict()
user_test_music = dict()

for u in music_data:
    user_valid_music[u] = [music_data[u][-2]]
    user_test_music[u] = [music_data[u][-1]]
    user_train_music[u] = music_data[u][:-2]

# of users: 7543,  # of items: 17835


In [10]:
count = 0
for i in playlists:
    count += len(i)

In [12]:
1 - count / num_users / num_items

0.9983535792788201

In [17]:
top_k = 500

In [18]:
'''
TransRec
'''
from models.TransRec_sequential import TransRec_sequential

seed_everything(seed)
TransRec = TransRec_sequential(user_train_music, user_valid_music, user_num=num_users, item_num=num_items, emb_dim=200, maxlen=1,
                                num_epochs=100, eval_every=1, early_stop_trial=5, learning_rate=0.001, reg_lambda=0.0, batch_size=128, device=device, top_k= 500)
TransRec.fit()
TransRec_ndcg, TransRec_recall = eval_sequential(TransRec, user_train_music, user_valid_music, user_test_music, num_users, num_items, top_k, mode='test')
print(f"\n[TransRec]\t Test_Recall@{top_k} = {TransRec_recall:.4f} Test_NDCG@{top_k} = {TransRec_ndcg:.4f}")
print("parmas :", sum(p.numel() for p in TransRec.parameters() if p.requires_grad))
print("======================================")

valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 397.95it/s]


epoch 1 train_loss = 0.63407052 valid_recall@500 = 0.1650 valid_ndcg@500 = 0.0247


valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 345.52it/s]


epoch 2 train_loss = 0.59141773 valid_recall@500 = 0.1880 valid_ndcg@500 = 0.0301


valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 383.92it/s]


...

epoch 16 train_loss = 0.12022328 valid_recall@500 = 0.1930 valid_ndcg@500 = 0.0292


valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 309.56it/s]


Early stop at epoch:17


test: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7543/7543 [00:21<00:00, 357.03it/s]


[TransRec]	 Test_Recall@500 = 0.1757 Test_NDCG@500 = 0.0278
parmas : 5093635


In [19]:
"""
SASRec
"""
from models.SASRec_sequential import SASRec_sequential

seed_everything(seed)
SASRec = SASRec_sequential(user_train_music, user_valid_music, user_num=num_users, item_num=num_items, hidden_dim=50, maxlen=20, num_blocks=1, num_heads=1,
                            num_epochs=100, eval_every=1, early_stop_trial=5, learning_rate=0.001, reg_lambda=0.0, batch_size=128, device=device, top_k = 500)
SASRec.fit()
SASRec_ndcg, SASRec_recall = eval_sequential(SASRec, user_train_music, user_valid_music, user_test_music, num_users, num_items, top_k, mode='test')
print(f"\n[SASRec]\t Test_Recall@{top_k} = {SASRec_recall:.4f} Test_NDCG@{top_k} = {SASRec_ndcg:.4f}")
print("parmas :", sum(p.numel() for p in SASRec.parameters() if p.requires_grad))

valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 296.72it/s]


epoch 1 train_loss = 5.7797 valid_recall@500 = 0.0140 valid_ndcg@500 = 0.0019


valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 295.87it/s]


epoch 2 train_loss = 4.5742 valid_recall@500 = 0.0380 valid_ndcg@500 = 0.0054


valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 295.75it/s]


...

epoch 8 train_loss = 1.6995 valid_recall@500 = 0.0370 valid_ndcg@500 = 0.0050


valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 296.39it/s]


Early stop at epoch:9


test: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7543/7543 [00:25<00:00, 294.53it/s]


[SASRec]	 Test_Recall@500 = 0.0327 Test_NDCG@500 = 0.0047
parmas : 908400


In [20]:
"""
BERT4Rec
"""
from models.BERTRec_sequential import BERTRec_sequential

seed_everything(seed)
BERTRec = BERTRec_sequential(user_train_music, user_valid_music, user_num=num_users, item_num=num_items, hidden=50*5, maxlen=50, n_layers=2, heads=5, mask_prob=0.1,
                                num_epochs=100, eval_every=1, early_stop_trial=5, learning_rate=0.001, reg_lambda=0.0, batch_size=128, device=device, top_k =500)
BERTRec.fit()
BERTRec_ndcg, BERTRec_recall = eval_sequential(BERTRec, user_train_music, user_valid_music, user_test_music, num_users, num_items, top_k, mode='test')
print(f"\n[BERTRec]\t Test_Recall@{top_k} = {BERTRec_recall:.4f} Test_NDCG@{top_k} = {BERTRec_ndcg:.4f}")
print("parmas :", sum(p.numel() for p in BERTRec.parameters() if p.requires_grad))

valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 188.49it/s]


epoch 1 train_loss = 9.7569 valid_recall@500 = 0.1320 valid_ndcg@500 = 0.0191


valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 204.56it/s]


epoch 2 train_loss = 9.7260 valid_recall@500 = 0.1400 valid_ndcg@500 = 0.0220


valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 219.88it/s]


...

epoch 32 train_loss = 8.2482 valid_recall@500 = 0.3390 valid_ndcg@500 = 0.0586


valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 220.81it/s]


Early stop at epoch:33


test: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7543/7543 [00:31<00:00, 240.37it/s]


[BERTRec]	 Test_Recall@500 = 0.3263 Test_NDCG@500 = 0.0530
parmas : 10455086


In [21]:
train_rows = []
for user in user_train_music:
    for song in user_train_music[user]:
        train_rows.append((user,song,1))
train = pd.DataFrame(train_rows, columns = ['user_id', 'item_id', 'rating'])

In [22]:
valid_rows = []
for user in user_valid_music:
    for song in user_valid_music[user]:
        valid_rows.append((user,song,1))
valid = pd.DataFrame(valid_rows, columns = ['user_id', 'item_id', 'rating'])

In [23]:
test_rows = []
for user in user_test_music:
    for song in user_test_music[user]:
        test_rows.append((user,song,1))
test = pd.DataFrame(test_rows, columns = ['user_id', 'item_id', 'rating'])

In [24]:
train = train.to_numpy()
valid = valid.to_numpy()
test = test.to_numpy()

matrix = sparse.lil_matrix((num_users, num_items))
for (u, i, r) in train:
    matrix[u, i] = r
train = sparse.csr_matrix(matrix)

matrix = sparse.lil_matrix((num_users, num_items))
for (u, i, r) in valid:
    matrix[u, i] = r
valid = sparse.csr_matrix(matrix)

matrix = sparse.lil_matrix((num_users, num_items))
for (u, i, r) in test:
    matrix[u, i] = r
test = sparse.csr_matrix(matrix)

# train = train.toarray()
# valid = valid.toarray()
# test = test.toarray()

In [25]:
train = train+valid

In [26]:
top_k = 500

In [27]:
"""
Embarrassingly shallow autoencoders for sparse data, 
Harald Steck,
Arxiv.
"""
import os
import math
import numpy as np

class EASE_implicit():
    def __init__(self, train, valid, reg_lambda):
        self.train = train.toarray()
        self.valid = valid.toarray()
        self.num_users = train.shape[0]
        self.num_items = train.shape[1]
        self.reg_lambda = reg_lambda

    def fit(self):   
        train_matrix = self.train
        G = train_matrix.T @ train_matrix
        diag = np.diag_indices(G.shape[0])
        G[diag] += self.reg_lambda
        P = np.linalg.inv(G)
        

        self.enc_w = P / (-np.diag(P)) 
        self.enc_w[diag] = 0


        self.reconstructed = self.train @ self.enc_w

    def predict(self, user_id, item_ids):
        return self.reconstructed[user_id, item_ids]

In [28]:
"""
EASE
"""
# from models.EASE_implicit import EASE_implicit
from utils import eval_implicit
seed_everything(seed)
ease = EASE_implicit(train=train, valid=valid, reg_lambda=100)
ease.fit()

In [29]:
ease_prec, ease_recall, ease_ndcg = eval_implicit(ease, train.toarray(), test.toarray(), 500)
print("EASE Prec@100: %f, Recall@100: %f, NDCG@100: %f"%(ease_prec, ease_recall, ease_ndcg))

EASE Prec@100: 0.001033, Recall@100: 0.516638, NDCG@100: 0.120273
